# K-Fold-Crossvalidation with a VGG16

## To Do
- get validation process straight
- hyperparameter-tuning

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import shutil
print(os.listdir("/home/Deep_Learner/work/local/histopathologic-cancer-detection"))

from glob import glob 
from skimage.io import imread
import gc

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras import applications, regularizers, optimizers
from keras.utils import to_categorical

['test.zip', 'base_dir', 'test', 'train_labels.csv.zip', 'train.zip', 'sample_submission.csv.zip', 'train_labels.csv', 'train']


Using TensorFlow backend.


In [3]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

#fast plot of training history
def plot_history(history, modelname):
    hist_df = pd.DataFrame(history.history)
    fig, axs = plt.subplots(nrows=2, sharex=True, figsize=(16, 10))
    axs[0].plot(hist_df.val_acc, lw=5, label='Validation Accuracy')
    axs[0].plot(hist_df.acc, lw=5, label='Training Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].grid()
    axs[0].legend(loc=0)
    axs[1].plot(hist_df.val_loss, lw=5, label='Validation MLogLoss')
    axs[1].plot(hist_df.loss, lw=5, label='Training MLogLoss')
    axs[1].set_ylabel('MLogLoss')
    axs[1].set_xlabel('Epoch')
    axs[1].grid()
    axs[1].legend(loc=0)
    fig.savefig('History_{}.png' .format(modelname), dpi=300)
    plt.show();

from sklearn.metrics import roc_curve, roc_auc_score, auc#plotting the receiver operating characteristics --> evaluate performance cutting point vice
def plot_roc(label, predictions): #IDEA: set diffrent cutting point based on ROC for ensembling   
    roc_auc_score(label, predictions)
    print('The ROC-Score is: {}' .format(roc_auc_score))

    fpr_keras, tpr_keras, thresholds_keras = roc_curve(label, predictions)
    auc_keras = auc(fpr_keras, tpr_keras)
    #print(auc_keras)

    fig = plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_keras, tpr_keras, label='area = {:.3f}'.format(auc_keras))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve: {}' .format(auc_keras))
    plt.legend(loc='best')
    #fig.savefig('ROC-Curve.png', dpi=300) #saving PLOT 
    plt.show()
    
from sklearn.metrics import confusion_matrix
import itertools
# Source: Scikit Learn website
# http://scikit-learn.org/stable/auto_examples/
# model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-
# selection-plot-confusion-matrix-py
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

from scipy.misc import imread
#list of tiffs to array to plot_correct / incorrect-images + rounding predictions to compare to labels
def prep_im_label (val, y_pred):
    vals = []
    for i in val:
        vals.append(imread(i))
        #q = q+1
        #print(q)
        #print(i)
    vals = np.asarray(vals)    
    print(vals.shape)
    y_pred = np.round(y_pred)
    return(vals, y_pred)

#plotting correctly classified images: https://www.datacamp.com/community/tutorials/convolutional-neural-networks-python
def plot_correct(vals, y_pred, y_label, modelname):
    correct = np.where(y_pred==y_label)[0]
    print ("Found %d correct labels" % len(correct))


    fig, ax = plt.subplots(ncols=3, nrows=3)
    fig.subplots_adjust(hspace=0.5)
    plt.suptitle('Correct Images', fontsize=16)
    plt.figure(num=None, figsize=(50, 50), dpi=100, facecolor='w', edgecolor='k')

    for ax in ax.flatten():
        ax.axis('off')

    for i, correct in enumerate(correct[:9]):
        ax = fig.add_subplot(3,3,i+1)
        ax.imshow(vals[correct])
        ax.set_title("Predicted {}, Class {}".format(y_pred[correct], y_label[correct]), fontsize=10)
        fig.set_figheight(8)
        fig.set_figwidth(8)

    #fig.savefig('Correct_Images_{}.png' .format(modelname), dpi=100) #saving PLOT 

#Plotting incorrectly classified
def plot_incorrect(vals, y_pred, y_label, modelname):
    incorrect = np.where(y_pred!=y_label)[0]
    print ("Found %d incorrect labels" % len(incorrect))


    fig, ax = plt.subplots(ncols=3, nrows=3)
    fig.subplots_adjust(hspace=0.5)
    plt.suptitle('Incorrect Images', fontsize=16)
    plt.figure(num=None, figsize=(50, 50), dpi=100, facecolor='w', edgecolor='k')

    for ax in ax.flatten():
        ax.axis('off')

    for i, incorrect in enumerate(incorrect[:9]):
        ax = fig.add_subplot(3,3,i+1)
        ax.imshow(vals[incorrect])
        ax.set_title("Predicted {}, Class {}".format(y_pred[incorrect], y_label[incorrect]), fontsize=10)
        fig.set_figheight(8)
        fig.set_figwidth(8)

    #fig.savefig('Incorrect_Images._{}.png' .format(modelname), dpi=100) #saving PLOT 

In [13]:
#VGG-16 for heatmap generation 
img_width_VGG, img_height_VGG = 96, 96 #VGG
modelname = 'VGG'

from time import time
from keras.applications.nasnet import NASNetMobile, NASNetLarge, preprocess_input
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint, Callback, LambdaCallback
from keras.layers import Convolution1D, concatenate, SpatialDropout1D, GlobalMaxPool1D, GlobalAvgPool1D, Embedding, \
    Conv2D, SeparableConv1D, Add, BatchNormalization, Activation, GlobalAveragePooling2D, LeakyReLU, Flatten
from keras.layers import Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, \
    Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D
from keras.losses import mae, sparse_categorical_crossentropy, binary_crossentropy
from keras.models import Model
from keras.optimizers import Adam, RMSprop


def get_model_classif_VGG_base_nottrainable():
    base_model_VGG = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width_VGG, img_height_VGG, 3))
    print('Model loaded.')
    
    for layer in base_model_VGG.layers:
        layer.trainable = False
    #    print("trainable:", layer.name)
    
    x = base_model_VGG.output
    x = GlobalMaxPooling2D()(x)
    x = (Dense(256, activation='relu'))(x)
    x = (Dropout(0.5)) (x)
    predictions = (Dense(1, activation='sigmoid'))(x)
    model_VGG = Model(inputs=base_model_VGG.input, outputs=predictions)

    adam = optimizers.Adam(lr=0.0005)
    model_VGG.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['acc'])

    model_VGG.summary()
    return model_VGG

def get_model_classif_VGG_base_trainable():
    base_model_VGG = applications.VGG16(weights=None, include_top=False, input_shape=(img_width_VGG, img_height_VGG, 3))
    print('Model loaded.')
    
    for layer in base_model_VGG.layers:
        layer.trainable = True
    #    print("trainable:", layer.name)
    
    x = base_model_VGG.output
    x = GlobalMaxPooling2D()(x)
    x = (Dense(256, activation='relu'))(x)
    x = (Dropout(0.5)) (x)
    predictions = (Dense(1, activation='sigmoid'))(x)
    model_VGG = Model(inputs=base_model_VGG.input, outputs=predictions)

    adam = optimizers.Adam(lr=0.0001)
    model_VGG.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['acc'])

    model_VGG.summary()
    return model_VGG

def get_callbacks(name_weights):
    ReduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, mode='auto', cooldown=5, min_lr=0.00005)
    earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1, mode='auto')
    tensorboard_callback = TensorBoard(log_dir="logs_NasnetM/{}".format(time()), 
            histogram_freq=0, 
            write_graph=True, write_grads=False, write_images=True)
    checkpoint = ModelCheckpoint(name_weights, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
    return [earlystopping, ReduceLR, tensorboard_callback, checkpoint]

In [5]:
'''#visualize Augmentation from custom datagen!
def looking_at_augmentation (data_generator, batchsize):
    aug_gen = datagen.flow(X, y, batch_size = batchsize)

    im, label = next(aug_gen)  
    im = (im - np.min(im))/np.ptp(im) # to normalize all images --> matplotlib only takes pos. values between 0..1 / 0..255 
    imgs = list(im)
    labels = list(label)
    
    fig, ax = plt.subplots(ncols=3, nrows=3)
    fig.subplots_adjust(hspace=0.5)
    plt.suptitle('Augmented Images', fontsize=16)
    plt.figure(num=None, figsize=(50, 50), dpi=100, facecolor='w', edgecolor='k')

    for ax in ax.flatten():
        ax.axis('off')
    
    i = 0 
    for im, l  in zip(imgs, labels):
        ax = fig.add_subplot(3,3,i+1)
        ax.imshow(im)
        i = i+1
        ax.set_title("Label: {}".format(l))
        fig.set_figheight(8)
        fig.set_figwidth(8)
        if i == batchsize:
            break

    #fig.tight_layout()
    #fig.savefig('Augmented-Images.png', dpi=100)

looking_at_augmentation(datagen, batchsize=9)'''

'#visualize Augmentation from custom datagen!\ndef looking_at_augmentation (data_generator, batchsize):\n    aug_gen = datagen.flow(X, y, batch_size = batchsize)\n\n    im, label = next(aug_gen)  \n    im = (im - np.min(im))/np.ptp(im) # to normalize all images --> matplotlib only takes pos. values between 0..1 / 0..255 \n    imgs = list(im)\n    labels = list(label)\n    \n    fig, ax = plt.subplots(ncols=3, nrows=3)\n    fig.subplots_adjust(hspace=0.5)\n    plt.suptitle(\'Augmented Images\', fontsize=16)\n    plt.figure(num=None, figsize=(50, 50), dpi=100, facecolor=\'w\', edgecolor=\'k\')\n\n    for ax in ax.flatten():\n        ax.axis(\'off\')\n    \n    i = 0 \n    for im, l  in zip(imgs, labels):\n        ax = fig.add_subplot(3,3,i+1)\n        ax.imshow(im)\n        i = i+1\n        ax.set_title("Label: {}".format(l))\n        fig.set_figheight(8)\n        fig.set_figwidth(8)\n        if i == batchsize:\n            break\n\n    #fig.tight_layout()\n    #fig.savefig(\'Augmented-I

In [6]:
IMAGE_SIZE = 96
IMAGE_CHANNELS = 3

SAMPLE_SIZE = 80000
file_path = ('/home/Deep_Learner/work/local/histopathologic-cancer-detection/')

# Setup the data_frame: Path, ID, Label, Image
training_dir = file_path + 'train/'
data_frame = pd.DataFrame({'path': glob(os.path.join(training_dir,'*.tif'))})#getting the path into df
data_frame['id'] = data_frame.path.map(lambda x: x.split('/')[7].split('.')[0]) #splitting the id-name(the 8th / in the path name) out of path

labels = pd.read_csv(file_path + 'train_labels.csv')#reading labels

data_frame = data_frame.merge(labels, on = 'id')#merging labels with df
negatives = data_frame[data_frame.label == 0].sample(SAMPLE_SIZE)
positives = data_frame[data_frame.label == 1].sample(SAMPLE_SIZE)

data_frame = pd.concat([negatives, positives]).reset_index()

data_frame = data_frame[['path', 'id', 'label']]
data_frame['image'] = data_frame['path'].map(imread)

data_frame = shuffle(data_frame)
data_frame.head

<bound method NDFrame.head of                                                      path  \
10621   /home/Deep_Learner/work/local/histopathologic-...   
53943   /home/Deep_Learner/work/local/histopathologic-...   
65143   /home/Deep_Learner/work/local/histopathologic-...   
122435  /home/Deep_Learner/work/local/histopathologic-...   
25075   /home/Deep_Learner/work/local/histopathologic-...   
93930   /home/Deep_Learner/work/local/histopathologic-...   
130915  /home/Deep_Learner/work/local/histopathologic-...   
3204    /home/Deep_Learner/work/local/histopathologic-...   
130841  /home/Deep_Learner/work/local/histopathologic-...   
144100  /home/Deep_Learner/work/local/histopathologic-...   
77764   /home/Deep_Learner/work/local/histopathologic-...   
118893  /home/Deep_Learner/work/local/histopathologic-...   
29054   /home/Deep_Learner/work/local/histopathologic-...   
156158  /home/Deep_Learner/work/local/histopathologic-...   
35634   /home/Deep_Learner/work/local/histopathologic-.

In [7]:
#initialize StratifiedKFold
from sklearn.model_selection import StratifiedKFold
kfold = 4
skf = StratifiedKFold(n_splits=kfold, random_state=101, shuffle = False)

In [8]:
#prepare data for stratified-k-fold
imgs = []
for img in data_frame['image']:
    imgs.append(img)
        
X = np.asarray(imgs)
y = np.asarray(data_frame['label'])

In [9]:
print(X.shape)
print(y)
print(y.shape)

(160000, 96, 96, 3)
[0 0 0 ... 0 0 0]
(160000,)


In [10]:
#Dataaugmentation
import imgaug as ia
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.4, aug)
seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.5), # vertically flip 20% of all images
        sometimes(iaa.Affine(
            scale={"x": (0.9, 1.2), "y": (0.9, 1.2)}, #>0.9 will cut part of img in combination with translate
            translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, # >20% will also cut part of img
            rotate=(-10, 10), # 45/-45° -> works good with scale + translate to prevent cuts
            #shear=(-5, 5), # shear by -16 to +16 degrees
            mode=ia.ALL 
        )),
        iaa.SomeOf((0, 3), [
                sometimes(iaa.Superpixels(p_replace=(0.2, 0.5), n_segments=(10, 100))), #superpixel-representation --> better basallamina representation 
                iaa.OneOf([
                    iaa.GaussianBlur((0, 0.1)), #small blur effects --> better representation
                    #iaa.AverageBlur(k=(1, 3)), # k must be odd
                    #iaa.MedianBlur(k=(1, 3)), # 
                ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)), #cell wall represenation
                iaa.Emboss(alpha=(0, 0.8), strength=(0, 0.5)), #cell wall represenation
                #searching for edges or angles --> blobby mask --> better basallamina representation / nuclei
                iaa.SimplexNoiseAlpha(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0.2, 0.4)), #detects edges --> cell wall,..
                    iaa.DirectedEdgeDetect(alpha=(0.2, 0.4), direction=(0.0, 1.0)), #direction will make edges from random directions 
                ])),
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01*255), per_channel=0.2), # add gaussian noise to images
             iaa.OneOf([
                    iaa.Dropout((0.01, 0.03), per_channel=0.2), #rnd remove 5% in small pixels
                    iaa.CoarseDropout((0.01, 0.03), size_percent=(0.01, 0.02), per_channel=0.2),# rnd remove 3% in big pixels
                ]),
                #iaa.Invert(0.01, per_channel=True), # invert color channels
                #iaa.Add((-10, 10), per_channel=0.3), # change brightness of images (by -10 to 10 of original value)
                iaa.AddToHueAndSaturation((-0.1, 0.1)), # change hue and saturation
                #
                #either change the brightness of the whole image (sometimes per channel) or change the brightness of subareas
                iaa.OneOf([
                    #iaa.Multiply((0.9, 1.1), per_channel=0.5),
                    #iaa.FrequencyNoiseAlpha(
                        #exponent=(-1, 0),
                        #first=iaa.Multiply((0.9, 1.1), per_channel=True),
                        #second=iaa.ContrastNormalization((0.9, 1.1))
                    #)
                ]),
                sometimes(iaa.ElasticTransformation(alpha=(0, 0.5), sigma=0.1)), #still not sure: move pixels locally around
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.02))), #still not sure:move parts of the image around
                #sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
                     random_order=True
        )
    ],
    random_order=True
)

datagen = ImageDataGenerator(preprocessing_function=seq.augment_image)
datagen_test = ImageDataGenerator(horizontal_flip=False)

In [11]:
train_batch_size = 64
train_batch_size_2 = 256
val_batch_size = 64

In [14]:
#doesnt match the unshuffled numbers???
i=0
for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    print('[Fold %d/%d]' % (i + 1, kfold))
    i=i+1
    #slice training-data into folds for training/testing variables
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    #get .fit values
    num_train_samples = len(X_train)
    num_val_samples = len(X_valid)  
    train_steps = np.ceil(num_train_samples // train_batch_size)
    val_steps = np.ceil(num_val_samples // val_batch_size)
    
    #get data-generators
    train_generator = datagen.flow(X_train, y_train, batch_size = train_batch_size)
    val_generator = datagen_test.flow(X_valid, y_valid, batch_size = val_batch_size)
    
    #Callbacks & Model
    name_weights = (base_path + "\Fold_" + str(i) + "{}.h5" .format(model_name))
    callbacks_list = get_callbacks(name_weights = name_weights)
    model_VGG = get_model_classif_VGG_base_nottrainable()
 
    #first on small batch_size to establish the basic informations
    history = model_VGG.fit_generator(
        train_generator,
        epochs=5, verbose=1,
        validation_data = val_generator,
        steps_per_epoch=train_steps,
        validation_steps=val_steps,
        callbacks=callbacks_list)

    #second round --> whole model with larger batchsize
    num_train_samples = len(X_train)
    num_val_samples = len(X_valid)  
    train_steps = np.ceil(num_train_samples // train_batch_size_2)
    val_steps = np.ceil(num_val_samples // val_batch_size)
    
    #get data-generators
    train_generator = datagen.flow(X_train, y_train, batch_size = train_batch_size_2)
    val_generator = datagen_test.flow(X_valid, y_valid, batch_size = val_batch_size) 
    model_VGG_bt = get_model_classif_VGG_base_trainable()
    model_VGG_bt.load_weights(name_weights)
        
    history = model_VGG_bt.fit_generator(
        train_generator,
        epochs=50, verbose=1,
        validation_data = val_generator,
        steps_per_epoch=train_steps,
        validation_steps=val_steps,
        callbacks=callbacks_list)
    
    print(model_VGG.evaluate(X_valid, y_valid))

TRAIN: [ 39910  39916  39917 ... 159997 159998 159999] TEST: [    0     1     2 ... 40075 40078 40079]
[Fold 1/4]
Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
______________

MemoryError: 

In [ ]:
#Test-Time

In [ ]:
test_dir = file_path + '/test'
df_test = pd.DataFrame({'path': glob(os.path.join(test_dir,'*.tif'))})
df_test['id'] = df_test.path.map(lambda x: x.split('/')[8].split('.')[0])
df_test['image'] = df_test['path'].map(imread)

#make flow_from_dataframe work
datagen_test = ImageDataGenerator(horizontal_flip=False)
test_gen = datagen_test.flow_from_dataframe(df_test, 
                                            directory=test_dir, 
                                            x_col='id', 
                                            y_col=None, 
                                            has_ext=False, 
                                            batch_size = 1, 
                                            shuffle=False, 
                                            class_mode = None, 
                                            target_size = (IMAGE_SIZE,IMAGE_SIZE))

df_test.head()

In [ ]:
#another way of loading testdata
test_files = glob('/home/Deep_Learner/work/local/histopathologic-cancer-detection/test/*.tif')

In [ ]:
#instantiate 4 different models
model_NASNet_fold1 = get_model_classif_nasnetm()
model_NASNet_fold2 = get_model_classif_nasnetm()
model_NASNet_fold3 = get_model_classif_nasnetm()
model_NASNet_fold4 = get_model_classif_nasnetm()

#Loading different k_fold-states
model_NASNet_fold1.load_weights('Final_NASNetMobile_fold_1_weights.h5')
print('Model model_NASNet_fold1 loaded!')
model_NASNet_fold2.load_weights('Final_NASNetMobile_fold_2_weights.h5')
print('Model model_NASNet_fold2 loaded!')
model_NASNet_fold3.load_weights('Final_NASNetMobile_fold_3_weights.h5')
print('Model model_NASNet_fold3 loaded!')
model_NASNet_fold4.load_weights('Final_NASNetMobile_fold_4_weights.h5')
print('Model model_NASNet_fold4 loaded!')

In [ ]:
#short Validation to see if its working
print(model_NASNetM.evaluate(X_valid, y_valid))

In [ ]:
def get_id_from_file_path(file_path):
    return file_path.split(os.path.sep)[-1].replace('.tif', '')

#dataaugmentation from https://imgaug.readthedocs.io/en/latest/index.html
def chunker(seq, size): #slices input(seq) into batches with the size=batch_size
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [ ]:
batch_size = 128
preds = []
ids = []
for batch in chunker(test_files, batch_size):
    X = [preprocess_input(cv2.imread(x).astype(np.float32)) for x in batch]
    ids_batch = [get_id_from_file_path(x) for x in batch]
    X = np.array(X)
    preds_batch = ((model_NASNet_fold1.predict(X).ravel()*model_NASNet_fold2.predict(X[:, ::-1, :, :]).ravel()*model_NASNet_fold3.predict(X[:, ::-1, ::-1, :]).ravel()*model_NASNet_fold4.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist()
    preds += preds_batch
    ids += ids_batch
    print(len(preds))

In [ ]:
df = pd.DataFrame({'id':ids, 'label':preds})
df.to_csv("baseline0_k_fold_crossvalidation.csv", index=False)
df.head()
print(len(df))

In [ ]:
!kaggle competitions submit -c histopathologic-cancer-detection -f baseline0_k_fold_crossvalidation.csv -m "k-fold-crossvalidation NASNetM"

In [ ]:
# Setup the data_frame: Path, ID, Label, Image
crossval_path = r"C:\Users\eg38emed\FCD\Tiles\Train_crossvalidation"

fcd = crossval_path + '\FCD'
tsc = crossval_path + '\TSC'

#fcd folder
df = pd.DataFrame({'path': glob(os.path.join(fcd,'*.png'))})
df['id'] = df.path.map(lambda x: x.split('\\')[6].split('.')[0]) 
df['label'] = 0

#tsc folder
df_pos = pd.DataFrame({'path': glob(os.path.join(tsc,'*.png'))})
df_pos['id'] = df_pos.path.map(lambda x: x.split('\\')[6].split('.')[0]) 
df_pos['label'] = 1

#both
df_train = pd.concat([df, df_pos])

#add images
df_train['image'] = df_train['path'].map(imread)

print(len(df_train))
print(df_train)

#initialize StratifiedKFold
from sklearn.model_selection import StratifiedKFold
kfold = 5
skf = StratifiedKFold(n_splits=kfold, random_state=101, shuffle = False)

train_batch_size = 64
val_batch_size = 64

#datagenerators
datagen_train = ImageDataGenerator(preprocessing_function=seq.augment_image,
                            rescale=1./255)

datagen_val = ImageDataGenerator(rescale=1./255)

#doesnt match the unshuffled numbers???
i=0
for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    print('[Fold %d/%d]' % (i + 1, kfold))
    i=i+1
    #slice training-data into folds for training/testing variables
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    #get .fit values
    num_train_samples = len(X_train)
    num_val_samples = len(X_valid)  
    train_steps = np.ceil(num_train_samples // train_batch_size)
    val_steps = np.ceil(num_val_samples // val_batch_size)
    
    #get data-generators
    train_generator = datagen_train.flow(X_train, y_train, batch_size = train_batch_size)
    val_generator = datagen_val.flow(X_valid, y_valid, batch_size = val_batch_size)
    
    #Callbacks & Model
    name_weights = (base_path + "\Fold_" + str(i) + "{}.h5" .format(model_name))
    callbacks_list = get_callbacks(name_weights = name_weights)
    model_VGG = get_model_classif_VGG_base_nottrainable()
 
    #first on small batch_size to establish the basic informations
    history = model_VGG.fit_generator(
        train_generator,
        epochs=5, verbose=1,
        validation_data = val_generator,
        steps_per_epoch=train_steps,
        validation_steps=val_steps,
        callbacks=callbacks_list)

    #second round -> whole model with larger batchsize
    num_train_samples = len(X_train)
    num_val_samples = len(X_valid)  
    train_steps = np.ceil(num_train_samples // train_batch_size)
    val_steps = np.ceil(num_val_samples // val_batch_size)
    
    #get data-generators
    train_generator = datagen_train.flow(X_train, y_train, batch_size = train_batch_size)
    val_generator = datagen_val.flow(X_valid, y_valid, batch_size = val_batch_size) 
    model_VGG_bt = get_model_classif_VGG_base_trainable()
    model_VGG_bt.load_weights(name_weights)
        
    history = model_VGG_bt.fit_generator(
        train_generator,
        epochs=50, verbose=1,
        validation_data = val_generator,
        steps_per_epoch=train_steps,
        validation_steps=val_steps,
        callbacks=callbacks_list)
    
    print(model_VGG.evaluate(X_valid, y_valid))